In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [5]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [11]:
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [7]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [8]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [12]:
# Get a list of column names and types
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])
# columns

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [14]:
# Get a list of column names and types
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])
# columns

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


# Exploratory Climate Analysis

In [15]:
# Earliest Date
session.query(Measurement.date).order_by(Measurement.date).first()

('2010-01-01')

In [16]:
# Latest Date
session.query(Measurement.date).order_by(Measurement.date.desc()).first()


('2017-08-23')

In [17]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Calculate the date 1 year ago from the last data point in the database

# Use Pandas Plotting with Matplotlib to plot the data


In [56]:
# Perform a query to retrieve the data and precipitation scores
# Save the query results as a Pandas DataFrame and set the index to the date column
# Sort the dataframe by date
oneyear_all = session.query(Measurement.date, Measurement.prcp, Measurement.station, Measurement.tobs).\
filter(Measurement.date >= query_date).\
order_by(Measurement.date).all()
oneyear_all_df = pd.DataFrame(oneyear_all, columns=['date','precipitation', 'station', 'tobs'])
oneyear_all_df.set_index(oneyear_all_df['date'], inplace=True)
oneyear_all

[('2016-08-23', 0.0, 'USC00519397', 81.0),
 ('2016-08-23', 0.15, 'USC00513117', 76.0),
 ('2016-08-23', 0.05, 'USC00514830', 80.0),
 ('2016-08-23', None, 'USC00517948', 80.0),
 ('2016-08-23', 0.02, 'USC00519523', 80.0),
 ('2016-08-23', 1.79, 'USC00519281', 77.0),
 ('2016-08-23', 0.7, 'USC00516128', 74.0),
 ('2016-08-24', 0.08, 'USC00519397', 79.0),
 ('2016-08-24', 2.15, 'USC00513117', 76.0),
 ('2016-08-24', 2.28, 'USC00514830', 80.0),
 ('2016-08-24', None, 'USC00517948', 78.0),
 ('2016-08-24', 1.22, 'USC00519523', 79.0),
 ('2016-08-24', 2.15, 'USC00519281', 77.0),
 ('2016-08-24', 1.45, 'USC00516128', 74.0),
 ('2016-08-25', 0.08, 'USC00519397', 80.0),
 ('2016-08-25', 0.08, 'USC00513117', 77.0),
 ('2016-08-25', 0.0, 'USC00514830', 81.0),
 ('2016-08-25', 0.0, 'USC00517948', 81.0),
 ('2016-08-25', 0.21, 'USC00519523', 80.0),
 ('2016-08-25', 0.06, 'USC00519281', 80.0),
 ('2016-08-25', 0.11, 'USC00516128', 77.0),
 ('2016-08-26', 0.0, 'USC00519397', 79.0),
 ('2016-08-26', 0.03, 'USC00513117', 

In [34]:
# Perform a query to retrieve the data and precipitation scores
# Save the query results as a Pandas DataFrame and set the index to the date column
# Sort the dataframe by date
#oneyear = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= query_date).\
#order_by(Measurement.date).all()
oneyear_df = pd.DataFrame(oneyear_all_df, columns=['date','precipitation'])
oneyear_df.set_index(oneyear_df['date'], inplace=True)
oneyear_df

,date,precipitation
date,,
2016-08-23,2016-08-23,0.00
2016-08-23,2016-08-23,0.15
2016-08-23,2016-08-23,0.05
2016-08-23,2016-08-23,NaN
2016-08-23,2016-08-23,0.02
2016-08-23,2016-08-23,1.79
2016-08-23,2016-08-23,0.70
2016-08-24,2016-08-24,0.08
2016-08-24,2016-08-24,2.15


In [18]:
# Calculate the date 1 year ago from the last data point in the database
query_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print("Query Date: ", query_date)

Query Date:  2016-08-23


![precipitation](Images/precipitation.png)

In [26]:
# Use Pandas to calcualte the summary statistics for the precipitation data
oneyear_df.describe()

,precipitation
count,2021.000000
mean,0.177279
std,0.461190
min,0.000000
25%,0.000000
50%,0.020000
75%,0.130000
max,6.700000


![describe](Images/describe.png)

In [63]:
# Design a query to show how many stations are available in this dataset?
unique_stations = session.query(Measurement.station).distinct().count()
unique_stations

9

In [66]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.

station_activity = session.query(Measurement.station, func.count(Measurement.prcp)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.prcp).desc()).all()
station_activity

[('USC00519281', 2772),
 ('USC00513117', 2696),
 ('USC00519397', 2685),
 ('USC00519523', 2572),
 ('USC00516128', 2484),
 ('USC00514830', 1937),
 ('USC00511918', 1932),
 ('USC00517948', 683),
 ('USC00518838', 342)]

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
station_activity=pd.DataFrame(oneyear_all_df, columns=['station', 'precipitation'])
group_statation= station_activity.groupby('station').count()
group_statation= group_statation.sort_values('precipitation', ascending=False)
group_statation

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


![precipitation](Images/station-histogram.png)

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
